## 자연어 처리 딥러닝
# Attention
- https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3
<img src='https://miro.medium.com/max/700/1*qN2Pj5J4VqAFf7dsA2dHpA.png' />

<img src='https://1.bp.blogspot.com/-AVGK0ApREtk/WaiAuzddKVI/AAAAAAAAB_A/WPV5ropBU-cxrcMpqJBFHg73K9NX4vywwCLcBGAs/s640/image2.png' />

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras

### seq2seq 모델에 적용

In [3]:
X_enc = np.random.randn(2,5,3).astype(np.float32) # np.float32
X_dec = np.random.randn(2,4,3).astype(np.float32) # np.float32

In [4]:
lstm_enc = keras.layers.LSTM(3, return_sequences=True, return_state=True)
lstm_dec = keras.layers.LSTM(3)

In [5]:
y, h, c = lstm_enc(X_enc)

y.shape, h.shape, c.shape

(TensorShape([2, 5, 3]), TensorShape([2, 3]), TensorShape([2, 3]))

- attention 계산

In [11]:
W = keras.layers.Dense(3) # 인코더의 출력값(y)에 곱해준다

In [12]:
score = tf.matmul(X_dec, W(y), transpose_b=True)

score.shape

TensorShape([2, 4, 5])

In [13]:
alignment = tf.math.softmax(score)
alignment.shape

TensorShape([2, 4, 5])

In [14]:
context = tf.matmul(alignment, y)
context.shape

TensorShape([2, 4, 3])

- 디코더 입력값에 context 를 붙여서 디코더에 입력한다

In [15]:
x = tf.concat([X_dec, context], axis=-1)
x.shape

TensorShape([2, 4, 6])

In [16]:
result = lstm_dec(x, initial_state=[h,c])
result.shape

TensorShape([2, 3])

### keras.layers.Attention
- query : 디코더 입력값
- key : 인코더 상태값 (score 계산에 사용)
- value : 일반적으로 key 와 동일한 값 (context 계산에 사용)

In [17]:
att = keras.layers.Attention()

In [18]:
context2 = att([X_dec, y])
context2.shape

TensorShape([2, 4, 3])

- 직접 계산값과 비교 (W 를 적용하지 않음)

In [19]:
tf.matmul(tf.nn.softmax(tf.matmul(X_dec, y, transpose_b=True)), y)

<tf.Tensor: shape=(2, 4, 3), dtype=float32, numpy=
array([[[-0.10625303, -0.11075789, -0.08585775],
        [-0.13907228, -0.10923996, -0.06157244],
        [-0.11387325, -0.10999321, -0.06892427],
        [-0.11399966, -0.11031871, -0.07852577]],

       [[-0.0005033 ,  0.05401512,  0.03346653],
        [-0.00695398,  0.06522242,  0.03864483],
        [-0.0138316 ,  0.06081258,  0.07345709],
        [ 0.00128573,  0.05015335,  0.02353917]]], dtype=float32)>

In [20]:
context2

<tf.Tensor: shape=(2, 4, 3), dtype=float32, numpy=
array([[[-0.10625303, -0.11075789, -0.08585775],
        [-0.13907228, -0.10923996, -0.06157244],
        [-0.11387325, -0.10999321, -0.06892427],
        [-0.11399966, -0.11031871, -0.07852577]],

       [[-0.0005033 ,  0.05401512,  0.03346653],
        [-0.00695398,  0.06522242,  0.03864483],
        [-0.0138316 ,  0.06081258,  0.07345709],
        [ 0.00128573,  0.05015335,  0.02353917]]], dtype=float32)>

### keras.layers.MultiHeadAttention
- Attention is all you Need 논문 (Transformer)
<img src='http://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png' />

In [21]:
mhead = keras.layers.MultiHeadAttention(num_heads=2, key_dim=2)

In [22]:
context3, scores = mhead(X_dec, y, return_attention_scores=True)
context3.shape, scores.shape # scores -> (batch, mhead, dec, inc)

(TensorShape([2, 4, 3]), TensorShape([2, 2, 4, 5]))

- self-attention

In [23]:
context4, scores = mhead(X_dec, X_dec, return_attention_scores=True)
context4.shape, scores.shape # scores -> (batch, mhead, dec, inc)

(TensorShape([2, 4, 3]), TensorShape([2, 2, 4, 4]))

### 최신 언어 딥러닝 모델
- transformer : http://jalammar.github.io/illustrated-transformer/
- BERT : http://jalammar.github.io/illustrated-bert/
- GPT-3 : http://jalammar.github.io/how-gpt3-works-visualizations-animations/